# Model comparison



## Setup

In [7]:
import os
import sys
import dotenv
import json
from loguru import logger

# Append the models path in order to import the models
PROJECT_ROOT = os.path.join(os.getcwd(), 'src/')
print(PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)

# Load env variables
dotenv.load_dotenv()

# Set log level INFO
logger.remove()
logger.add(sys.stderr, level="INFO")

/home/twanh/workspace/thesis/thesis-advent-of-agents/src/


4

In [8]:
# Import models from the system
from models.base_model import BaseLanguageModel
from models.gemini_model import GeminiLanguageModel
from models.openai_model import OpenAILanguageModel
from models.deepseek_model import DeepseekLanguageModel

### Load puzzles and input/outputs

In [69]:
# Get the correct paths
test_data_folder = os.path.join(PROJECT_ROOT, '..', 'experiments', 'test_data')
puzzles_folder = os.path.join(test_data_folder, 'puzzles/')
input_output_file = os.path.join(test_data_folder, 'answers2024.json')
puzzle_files = [os.path.join(puzzles_folder, f) for f in os.listdir(puzzles_folder) if os.path.isfile(os.path.join(puzzles_folder, f))]

In [70]:
# Create a datastructure were we can get by day
json_data = {}
with open(input_output_file, 'r') as f:
    json_data = {item['day']: item for item in json.load(f)}

puzzle_data = []
for file_path in puzzle_files:
    # Get the day of the puzzle file
    file_name = os.path.basename(file_path)
    day_str = file_name.split('_')[-1].split('.')[0]
    day = int(day_str)

    if day in json_data:
        with open(file_path, 'r') as f:
            puzzle_description = f.read()

        puzzle_info = {
            "year": json_data[day]['year'],
            "day": day,
            "description": puzzle_description,
            "input": json_data[day]['input'],
            "expected_output": json_data[day]['part1']
        }

        puzzle_data.append(puzzle_info)

# Sort by day
puzzle_data.sort(key=lambda x: x['day'])
print(len(puzzle_data)) # should be 25

25


### Model Configurations

| Agent          | Configuration 1 (Ultimate Power) | Configuration 2 (Pre-AOC Power) | Configuration 3 (Reasoning Focus) | Configuration 4 (Speed Optimized) | Configuration 5 (P&D Reasoning) | Configuration 6 (Balanced Mix) |
| :------------- | :------------------------------ | :------------------------------ | :-------------------------------- | :------------------------------- | :------------------------------ | :----------------------------- |
| **preprocessing** | Claude 3.5 Sonnet               | Claude 3.5 Sonnet               | o3-mini (Reasoning)               | Claude 3.5 Haiku                  | GPT-4o mini                     | GPT-4o mini                     |
| **retrieval**  | Gemini 2.5 Pro                  | GPT-4.1                        | o4-mini (Reasoning)               | Gemini 2.0 Flash                 | Gemini 2.0 Flash                | Claude 3.5 Sonnet               |
| **planning**   | o3 (Reasoning)                  | o4-mini (Reasoning)            | o4-mini (Reasoning)               | GPT-4o mini                      | o4-mini (Reasoning)             | GPT-4.1                         |
| **coding**     | GPT-4.1                         | GPT-4.1                        | o3-mini (Reasoning)               | Gemini 2.0 Flash                 | GPT-4.1                         | Gemini 2.0 Flash                |
| **debugging**  | Claude 3.7 Sonnet               | o3-mini (Reasoning)            | o4-mini (Reasoning)               | o3-mini (Reasoning)              | o3-mini (Reasoning)             | o4-mini (Reasoning)             |

##### Ultimate Power
Aim: Achieve maximum possible performance by strategically deploying the most powerful models where they matter most.
Strategy: Places o3 (the most powerful reasoning model) in Planning where complex problem decomposition is critical. Uses elite models for each other role, pairing Claude 3.7 Sonnet's strong analytical abilities with debugging, and GPT-4.1's code generation excellence with the coding agent. Balanced across providers to leverage each company's strengths.

##### Pre-AOC Power
Aim: Maximize performance while controlling for potential AOC training data advantage.
Strategy: Deliberately excludes Gemini 2.5 models which may have seen AOC 2024 data. Still achieves high capability with Claude 3.5 Sonnet for preprocessing, GPT-4.1 for retrieval and coding, and reasoning-capable models strategically placed for planning (o4-mini) and debugging (o3-mini).

##### Reasoning Focus
Aim: Evaluate whether reasoning-flagged models throughout the entire pipeline significantly improve performance.
Strategy: Uses only models with explicit reasoning capabilities for all agent roles. Balances o3-mini (one use for coding) with o4-mini across other roles, creating a pipeline entirely focused on reasoning capabilities. Will reveal if reasoning throughout is worth the potential cost increase.

##### Speed Optimized
Aim: Maximize speed while maintaining robust performance on complex puzzles.
Strategy: Employs the fastest models in the collection (Haiku, Flash, mini variants) while reserving reasoning capabilities (o3-mini) only for debugging where errors are most critical to resolve quickly. Uses the speed of Gemini 2.0 Flash for coding while ensuring planning is still effective with GPT-4o mini.

#### P&D Reasoning
Aim: Test the hypothesis that reasoning capabilities mainly benefit Planning and Debugging.
Strategy: Concentrates reasoning models only in Planning and Debugging roles, using capable but non-reasoning models elsewhere. If this configuration performs nearly as well as the Reasoning Focus configuration but with lower cost/latency, it indicates that reasoning is most valuable in these specific roles.

#### Balanced Mix
Aim: Create a balanced, cost-effective pipeline using strengths from each provider.
Strategy: Distributes roles across providers to leverage each company's strengths: Google for fast coding, OpenAI for planning and debugging, and Claude for context-handling in retrieval. Avoids using the most expensive models while maintaining good coverage of capabilities across the pipeline.

In [81]:
from main import _get_model

# Configuration 1 -- Ultimate power
config_ultimate_power = {
    "preprocessing": _get_model("claude-3-5-sonnet-20241022"),
    "retrieval": _get_model("gemini-2.5-pro-preview-05-06"),
    "planning": _get_model("o3"),
    "coding": _get_model("gpt-4.1-2025-04-14"),
    "debugging": _get_model("claude-3-7-sonnet@20250219"),
}

# Configuration 2 -- PreAOC power
config_pre_aoc_power = {
    "preprocessing": _get_model("claude-3-5-sonnet-20241022"),
    "retrieval": _get_model("gpt-4.1-2025-04-14"),
    "planning": _get_model("o4-mini-2025-04-16"),
    "coding": _get_model("gpt-4.1-2025-04-14"),
    "debugging": _get_model("o3-mini-2025-01-31"),
}

# Configuration 3 -- Reasoning
config_reasoning_focus = {
    "preprocessing": _get_model("o3-mini-2025-01-31"),
    "retrieval": _get_model("o4-mini-2025-04-16"),
    "planning": _get_model("o4-mini-2025-04-16"),
    "coding": _get_model("o3-mini-2025-01-31"),
    "debugging": _get_model("o4-mini-2025-04-16"),
}

# Configuration 4 -- Speed
config_speed_optimized = {
    "preprocessing": _get_model("claude-3-5-haiku-20241022"),
    "retrieval": _get_model("gemini-2.0-flash"),
    "planning": _get_model("gpt-4o-mini-2024-07-18"),
    "coding": _get_model("gemini-2.0-flash"),
    "debugging": _get_model("o3-mini-2025-01-31"),
}

# Configuration 5 -- Planning and Debugging Reasoning
config_pd_reasoning = {
    "preprocessing": _get_model("gpt-4o-mini-2024-07-18"),
    "retrieval": _get_model("gemini-2.0-flash"),
    "planning": _get_model("o4-mini-2025-04-16"),
    "coding": _get_model("gpt-4.1-2025-04-14"),
    "debugging": _get_model("o3-mini-2025-01-31"),
}

# Configuration 6 -- Balanced
config_balanced_mix = {
    "preprocessing": _get_model("gpt-4o-mini-2024-07-18"),
    "retrieval": _get_model("claude-3-5-sonnet-20241022"),
    "planning": _get_model("gpt-4.1-2025-04-14"),
    "coding": _get_model("gemini-2.0-flash"),
    "debugging": _get_model("o4-mini-2025-04-16"),
}

configurations_to_test = {
#    "ultimate_power": config_ultimate_power, TEMPORARY DISABLED BECAUSE IT RAN ALREADY
    "pre_aoc_power": config_pre_aoc_power,
    "reasoning_focus": config_reasoning_focus,
    "speed_optimized": config_speed_optimized,
    "pd_reasoning": config_pd_reasoning,
    "balanced_mix": config_balanced_mix,
}


## System

In [72]:
from agents.base_agent import BaseAgent
from agents.coding_agent import CodingAgent
from agents.debugging_agent import DebuggingAgent
from agents.planning_agent import PlanningAgent
from agents.pre_processing_agent import PreProcessingAgent
from agents.retreival_agent import RetrievalAgent
from core.orchestrator import Orchestrator
from utils.util_types import AgentSettings
from core.state import MainState
from utils.util_types import Puzzle

In [73]:

def setup_system(config: dict[str, BaseLanguageModel], expected_output: str, puzzle_input: str):

    preprocessing_model = config["preprocessing"]
    retrieval_model = config["retrieval"]
    planning_model = config["planning"]
    coding_model = config["coding"]
    debugging_model = config["debugging"]

    agents = (
        (
            PreProcessingAgent(
                'preprocess', model=preprocessing_model,
            ),
            AgentSettings(enabled=True, can_debug=False),
        ),
        (
            RetrievalAgent(
                'retreival',
                model=retrieval_model,
                connection_string=os.getenv('DB_CONNECTION_STRING') or '',
                openai_key=os.getenv('OPENAI_API_KEY') or '',
                weights=None, # Use default weights
            ),
            AgentSettings(enabled=True, can_debug=False),
        ),
        (
            PlanningAgent(
                'planning',
                model=planning_model,
                n_plans=3, # Keep n_plans fixed for consistent comparison
            ),
            AgentSettings(enabled=True, can_debug=False),
        ),
        (
            CodingAgent('coding', model=coding_model),
            AgentSettings(enabled=True, can_debug=False),
        ),
        (
            DebuggingAgent(
                'debugging',
                model=debugging_model,
                expected_output=expected_output,
                puzzle_input=puzzle_input,
            ),
            AgentSettings(enabled=True, can_debug=True),
        ),
    )

    orchestrator = Orchestrator(agents, {})
    return orchestrator


In [ ]:
import time
def run_and_test_system(
    day: int,
    puzzle_desc: str,
    puzzle_input: str,
    expected_output: str,
    config: dict 
) -> dict[str, str|int|None]:

    orchestrator = setup_system(
        config,
        expected_output=expected_output,
        puzzle_input=puzzle_input
    )


    puzzle = Puzzle(
        description=puzzle_desc,
        solution=None,
        year=2024,
        day=day,
    )

    state = MainState(puzzle=puzzle)

    try:
        start_time = time.time()
        ret_state = orchestrator.solve_puzzle(state)
        end_time = time.time()
        return {
            'success': ret_state.is_solved,
            'day': day,
            'code': ret_state.final_code,
            'debug_attempts': ret_state.debug_attempts,
            'debug_suggestions': ret_state.debug_suggestions,
            'n_retreived_puzzles': len(ret_state.retreived_puzzles),
            'keywords': ','.join(ret_state.keywords),
            'concepts': ','.join(ret_state.underlying_concepts),
            'time': end_time - start_time
        }

    except Exception as e:
        print(f"Runtime error during puzzle solving for Day {day}: {e}")
        return {
            'success': False,
            'code': None,
            'debug_attempts': None,
            'debug_suggestions': None,
            'n_retreived_puzzles': None,
            'keywords': None,
            'concepts': None,
            'time': None,
        }

In [75]:
import datetime
import pandas as pd
SAVE_DIR = os.path.join(PROJECT_ROOT, '../', 'experiments/', 'results/', 'model_comparison/')

def run_config(config: dict[str, BaseLanguageModel], config_name: str, puzzle_data):

    all_results = []
    for puzzle in puzzle_data:
        # Load puzzle
        puzzle_day = puzzle['day']
        puzzle_description = puzzle['description']
        input_ = puzzle['input']
        expected_ouptut = puzzle['expected_output']
        print(f"Running day {puzzle_day}")

        # RUn the tests
        results = run_and_test_system(
            puzzle_day,
            puzzle_description,
            input_,
            expected_ouptut,
            config,
        )

        results['config_name'] = config_name
        all_results.append(results)

    # Save the resu
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f'comp-{config_name}-{timestamp}.csv'
    filepath =os.path.join(SAVE_DIR, filename)

    results_df = pd.DataFrame(all_results)
    print(f"Saving results for {config_name} to {filepath}")
    results_df.to_csv(filepath, index=False)
    
    return results_df

In [ ]:
# Runn all configs

results_list = []
for config_name, config in configurations_to_test.items():

    print(f"----- Running config {config_name}")
    print(config)
    results_config = run_config(config, config_name, puzzle_data)
    results_list.append(results_config)

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'all_configs-{timestamp}.csv'
filepath = os.path.join(SAVE_DIR, filename)
results = pd.concat(results_list)
results.to_csv(filepath, index=False)

----- Running config pre_aoc_power
{'preprocessing': <models.anthropic_model.AnthropicLanguageModel object at 0x704d852b96d0>, 'retrieval': <models.openai_model.OpenAILanguageModel object at 0x704d852bb1d0>, 'planning': <models.openai_model.OpenAILanguageModel object at 0x704d852dea80>, 'coding': <models.openai_model.OpenAILanguageModel object at 0x704d852d3560>, 'debugging': <models.openai_model.OpenAILanguageModel object at 0x704d852c29f0>}
Running day 1


2025-05-20 20:36:10.090 | INFO     | core.retreival:init_db:154 - Database initialization complete.
2025-05-20 20:36:10.103 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: preprocess
2025-05-20 20:36:20.064 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: retreival
2025-05-20 20:37:08.787 | WARNING  | agents.retreival_agent:process:130 - Could not decode json: e=JSONDecodeError('Expecting property name enclosed in double quotes: line 8 column 5 (char 1739)')
2025-05-20 20:37:08.789 | WARNING  | agents.base_agent:_invalid_response_retry:66 - Retrying 1/3 for retreival
2025-05-20 20:38:06.208 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: planning
2025-05-20 20:38:06.213 | INFO     | agents.planning_agent:process:149 - Generating 3 plans
2025-05-20 20:38:06.215 | INFO     | agents.planning_agent:process:152 - Creating plan 1/3
2025-05-20 20:38:21.283 | INFO     | agents.planning_agent:process:152 - Creating plan 2/3
2025-05-20 20:38:44.117 

Running day 2


2025-05-20 20:39:34.152 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: retreival
2025-05-20 20:40:27.039 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: planning
2025-05-20 20:40:27.042 | INFO     | agents.planning_agent:process:149 - Generating 3 plans
2025-05-20 20:40:27.043 | INFO     | agents.planning_agent:process:152 - Creating plan 1/3
2025-05-20 20:40:55.090 | INFO     | agents.planning_agent:process:152 - Creating plan 2/3
2025-05-20 20:41:20.383 | INFO     | agents.planning_agent:process:152 - Creating plan 3/3
2025-05-20 20:41:39.274 | WARNING  | agents.planning_agent:_generate_solution_plan:98 - Could not extract markdown plan from response.generated_plan=[] ret='```markdown\n## 1. Overall Approach\nIterate over each report (one per line), parse it into a list of integers, compute the list of adjacent differences, then check whether those differences are all strictly positive (and ≤ 3) or all strictly negative (and ≥ −3). Count how many repo

Running day 3


...

2025-05-20 21:36:06.745 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: retreival
2025-05-20 21:36:10.903 | WARNING  | agents.retreival_agent:process:81 - Did not find any solutions for puzzle24-2016. Skipping...
2025-05-20 21:36:43.417 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: planning
2025-05-20 21:36:43.422 | INFO     | agents.planning_agent:process:149 - Generating 3 plans
2025-05-20 21:36:43.425 | INFO     | agents.planning_agent:process:152 - Creating plan 1/3
2025-05-20 21:37:02.222 | INFO     | agents.planning_agent:process:152 - Creating plan 2/3
2025-05-20 21:37:26.104 | INFO     | agents.planning_agent:process:152 - Creating plan 3/3
2025-05-20 21:38:00.412 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: coding
2025-05-20 21:38:19.791 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: debugging
2025-05-20 21:38:19.794 | INFO     | agents.debugging_agent:_run_test:337 - Running code with test case
2025-05-20 2

Running day 19


2025-05-20 21:38:28.980 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: retreival
2025-05-20 21:39:09.060 | WARNING  | agents.retreival_agent:process:81 - Did not find any solutions for puzzle16-2017. Skipping...
2025-05-20 21:39:09.063 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: planning
2025-05-20 21:39:09.065 | INFO     | agents.planning_agent:process:149 - Generating 3 plans
2025-05-20 21:39:09.065 | INFO     | agents.planning_agent:process:152 - Creating plan 1/3
2025-05-20 21:39:30.323 | INFO     | agents.planning_agent:process:152 - Creating plan 2/3
2025-05-20 21:39:44.308 | WARNING  | agents.planning_agent:_generate_solution_plan:98 - Could not extract markdown plan from response.generated_plan=[] ret='```markdown\n## 1. Overall Approach\nFor each desired design string, perform a classic "word‐break" dynamic programming check against the set of available patterns. Maintain a boolean DP array where `dp[i]` is true if the prefix of length `i` 

Running day 20


2025-05-20 21:40:51.403 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: retreival
2025-05-20 21:40:54.043 | WARNING  | agents.retreival_agent:process:81 - Did not find any solutions for puzzle13-2016. Skipping...
2025-05-20 21:41:46.313 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: planning
2025-05-20 21:41:46.318 | INFO     | agents.planning_agent:process:149 - Generating 3 plans
2025-05-20 21:41:46.320 | INFO     | agents.planning_agent:process:152 - Creating plan 1/3
2025-05-20 21:42:10.163 | WARNING  | agents.planning_agent:_generate_solution_plan:98 - Could not extract markdown plan from response.generated_plan=[] ret="```markdown\n## 1. Overall Approach\nPrecompute the shortest “normal” path distances between every pair of track cells (cells marked `.` or `S` or `E`) using BFS on the grid respecting walls. Then, for each track cell `i`, enumerate all track cells `j` reachable in exactly two moves when ignoring walls (a “cheat” segment). For each 